In [ ]:
import numpy as np
import torch

from torch.nn import functional as F
from torch.utils.data import DataLoader
import util
import geometry
import cv2
import os 
import pdb
import matplotlib.pyplot as plt
from pyntcloud import PyntCloud
import pandas as pd
import models
import hdf5_dataio
import glob
import data_util
from skimage.metrics import structural_similarity

import synthesize_novel_views

import wandb
%load_ext autoreload
%autoreload 2

In [ ]:
checkpoint_path = 'YOUR MODEL CHECKPOINT PATH'
network = 'relu' ##model parameters
conditioning = 'hyper' ## model parameter
data_root = 'LOCATION THE DATASET in hdf5 format'
data_root_images = 'LOCATION OF THE DATASET (IMAGES)'
img_sidelength = 128 ## model parameter
max_num_instances = None ## model parameter
specific_observation_idcs = [44] ## model parameter

In [ ]:
model = models.LFAutoDecoder(num_instances=2154, latent_dim=256, parameterization='plucker', network=network,
                             conditioning=conditioning)

In [ ]:
state_dict = torch.load(checkpoint_path)
num_instances = state_dict['latent_codes.weight'].shape[0]

In [ ]:
model.load_state_dict(state_dict)
model.eval()

In [ ]:
dataset = hdf5_dataio.SceneClassDataset(num_context=0,
                                                data_root=data_root,
                                                img_sidelength=img_sidelength,
                                                num_trgt=1,
                                                max_num_instances=max_num_instances,
                                                specific_observation_idcs=[44])

In [ ]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True,
                                  drop_last=True, num_workers=0)

In [ ]:
print(f"the number of batches {len(dataloader)}")

In [ ]:
first_batch = iter(dataloader).next()
q0, gt = first_batch

In [ ]:
"""
    Obtain some random poses, the model will synthesize theses poses based on an image that does not correspond to that pose
    e.g. 
    image corresponds to front of the car, pose correspond to the side of the car
    
    output => image of side of the car
"""

poses, pose_image_ids = synthesize_novel_views.pose_predictor_information(q0['query'], dataset_location) ## dataset location should point to where the images are...

In [ ]:
synthesize_novel_views.pose_predictor(model,q0,poses, pose_image_ids, novel_views_output_dir) ## novel views output dir should point to where you would like the output to be stored